# Progetto Computer Science

*Nome:* Christian

*Cognome:* Uccheddu

*Matricola:* 800428


**Setup dell'ambiente**

Importazione di tutte le librerie utili al funzionamento del programma e settaggio dei path in cui sono presenti i dati in modo da essere facilmente cambiabili all'inizio del programma.

In [63]:
import pandas as pd
import datetime
import numpy as np

In [64]:
loans_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans.csv"
loans_lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans_lenders.csv"
lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/lenders.csv"
country_stats_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/country_stats.csv" 

**1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**

Per questo compito mi occupo di importare il file loans_lenders, di analizzarne la struttura e successivamente decidere come procedere.

In [65]:
loans_lenders = pd.read_csv(loans_lenders_path)

In [66]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Vedendo la struttura del dataframe, la normalizzazione della tabella consiste nel separare gli elementi presenti nella colonna lenders in modo che il dataframe sia formato da celle in cui è presente solo una istanza. Per farlo separo gli elementi che si trovano in corrispondenza della virgola, essendoci uno spazio vuoto dopo la virgola dovrò stare attento a considerarlo. Per ovviare a questo problema trasformo gli elementi ad ogni istanza di lenders in una lista.

In [ ]:
loans_lenders['lenders'] = loans_lenders.apply(lambda row: row['lenders'].split(','), axis = 1)

Per separare gli elementi utilizzo la funzione *explode* implementata all'interno della libreria pandas che permette di sepoarare gli elementi di una lista lungo l'asse voluto.

In [ ]:
loans_lenders_norm = loans_lenders.explode('lenders')

In [ ]:
# Decommentare questa riga per vedere i risultati.
loans_lenders_norm.head()

**2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.**

Come per il punto precedente procedo con una esplorazione preliminare del file in cui sono presenti gli attributi utili allo svolgimento del compito. Dopo averne fatto una prima visualizzazione degli attributi si nota che è utile passare gli attributi "disburse_time" e "planned_expiration_time" come date al momento dell'importazione.

In [ ]:
loans = pd.read_csv(loans_path, parse_dates=["disburse_time", "planned_expiration_time"])

In [ ]:
loans.head()

In [ ]:
loans["duration"] = loans["planned_expiration_time"] - loans["disburse_time"]

Controllo per sicurezza quanti sono gli attributi nulli una volta fatta la differenza. In particolare è un numero abbastanza grosso, bisognerebbe quindi occuparsi di riempire i missing values.

In [ ]:
loans.loc[loans['duration'].isnull() == True]

**3) Find the lenders that have funded at least twice.**

Per fare questa cosa ho gli attributi già pronti. Per prima cosa bisogna effettuare un merge tra loans_lenders_norm e loans per sapere se tutte le istanze presenti in loans_lenders norm corrispondano allo stato "funded", una volta fatto questo mi occupo di tenere solo le righe in cui lo stato è funded e di selezionare solo quelle con un conteggio maggiore di o uguale a 2. Per non sovraccaricare troppo i conti mi preoccupo di fare il merge solo con gli attributi che serviranno durante i conti.

In [ ]:
loans_lenders_merged = pd.merge(loans_lenders_norm, loans[["loan_id", "status"]], on="loan_id", how="left")

In [ ]:
loans_lenders_merged_funded = loans_lenders_merged[loans_lenders_merged["status"] == "funded"]

In [ ]:
num_loan = loans_lenders_merged_funded.groupby('lenders').count()['loan_id']
num_loan.loc[num_loan >= 2]

**4) For each country, compute how many loans have involved that country as borrowers.**

Durante l'esecuzione di questo punto nasce la problematica causata dal gran numero di NaN presenti nell'attributo duration notato durante l'esecuzione del punto 2. Per effettuare un calcolo corretto bisogna quindi mettere delle condizioni, in particolare creo un nuovo dataframe che contenga le righe di loans con le seguenti caratteristiche:

- Non hanno valori null in "duration".
- Per quanto riguarda i valori in "planned_expiration_time" che siano minori di "disburse_time" vengono cancellati visto che sono pochi (circa 1\% come mostrato sotto). 

In [ ]:
loans_not_null = loans[loans["duration"].isnull() == False]
condition = loans_not_null[loans_not_null["planned_expiration_time"] < loans_not_null["disburse_time"]]
round(len(condition)/len(loans_not_null)*100,2)

In [ ]:
loans_not_null = loans_not_null[loans_not_null["planned_expiration_time"] > loans_not_null["disburse_time"]]

Ora mi occuppo di raggruppare per country e di contare solo quelli corretti.

In [ ]:
num_loans_country = loans_not_null.groupby('country_name').count()['loan_id']

In [ ]:
#Decommentare questa riga per la visualizzazione dei risultati.
#num_loans_country

**5) For each country, compute the overall amount of money borrowed.** 

Per farlo è gia a disposizione il dataset sistemato, bisogna solo raggruppare e sommare per paese.

In [ ]:
borr_country = loans_not_null.groupby('country_name')['loan_amount'].sum()
borr_country = borr_country.reset_index().rename(columns={"loan_amount" : "borrowed_amount"})

In [ ]:
#Decommentare questa riga per vedere i risultati
#borr_country

**6) Like the previous point, but expressed as a percentage of the overall amount lent.**

Allo stesso modo del punto precedente, basta stare attenti a fare il conto giusto.

In [ ]:
overall_amount_lent = loans_not_null['loan_amount'].sum()
borr_country_perc = pd.DataFrame(loans_not_null.groupby('country_name')['loan_amount'].sum())
borr_country_perc['percentage'] = round(borr_country_perc['loan_amount'] / overall_amount_lent * 100, 2)
borr_country_perc = borr_country_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

In [ ]:
#Decommentare questa riga per vedere i risultati.
#borr_country_perc

Eseguo la somma per vedere che il risultato è unitario, ciò è vero a meno di approssimazioni che sono state svolte durante i conteggi:

In [ ]:
round(borr_country_perc['percentage'].sum(),2)

**7) Like the three previous points, but split for each year (with respect to disburse time).**

Il primo passaggio è quello di riprendere il dataset creato in precedenza e settare l'attributo "disburse_time" come indice in modo da poterci applicare il metodo Grouper di pandas.

In [ ]:
loans_not_null["disburse_time"] =  pd.to_datetime(loans_not_null["disburse_time"])
loans_not_null = loans_not_null.set_index("disburse_time")
loans_by_year_sum = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_amount"].sum().to_frame()

Ora faccio le stesse cose fatte sopra ma calcolando la percentuale. Per farlo devo contare quanti sono stati e dividere il count per il "loan_amount" per il conteggio.

In [ ]:
loans_by_year_perc = loans_by_year_sum
loans_by_year_perc["loan_amount"] = loans_by_year_perc["loan_amount"]/overall_amount_lent*100
loans_by_year_perc = loans_by_year_perc.rename(columns={"loan_amount" : "loan_amount_perc"})

Faccio come sopra in modo da poter contare questa volta.

In [ ]:
loans_by_year_count = loans_not_null.groupby(["country_name", pd.Grouper(freq="Y")])["loan_id"].count().to_frame()
loans_by_year_count = loans_by_year_count.rename(columns={"loan_id" : "count"})

Eseguo il merge tra i tre dataframe selezionando solo le colonne che non sono duplicate in modo da avere il minor spreco di memoria possibile.

In [ ]:
country_statistics_by_year = pd.concat([loans_by_year_count, loans_by_year_sum, loans_by_year_perc], axis=1)
country_statistics_by_year = country_statistics_by_year.loc[:, ~country_statistics_by_year.columns.duplicated()]

In [ ]:
#Decommentare questa riga per stampare i risultati.
country_statistics_by_year

**8) For each lender, compute the overall amount of money lent.**

La prima cosa da fare è capire per ogni id il numero di lenders coinvolti, e ciò viene fatto contando il numero di persone presenti nel loan_id. L'assunzione fatta è che ogni lenders si sia occupato in egual misura di ogni loan.

In [28]:
lenders_num = loans_lenders_norm.groupby("loan_id").count().reset_index().rename(columns={"lenders" : "lenders_count"})

In [29]:
lenders_num_details = pd.merge(lenders_num, loans_not_null, on="loan_id")[["loan_id", "lenders_count", "loan_amount"]]

In [30]:
lenders_num_details["amount_per_person"] = lenders_num_details["loan_amount"] / lenders_num_details["lenders_count"]

In [31]:
loans_lenders_merged = pd.merge(loans_lenders_norm, lenders_num_details, on="loan_id", how="left")

In [32]:
lenders_overall_lent = loans_lenders_merged.groupby("lenders")["amount_per_person"].sum().to_frame().reset_index()

In [33]:
#Decommentare questa riga per vedere i risultati
lenders_overall_lent

**9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.**

La prima cosa da fare è sistemare il dataframe in modo da averlo senza valori NaN. Una volta diviso il dataframe in queste due parti cerco di calcolare la distribuzione nel dataframe in cui tutto è conosciuto e la applico al secondo dataframe.

In [34]:
lenders = pd.read_csv(lenders_path)

In [35]:
lenders_notnull = lenders.loc[lenders["country_code"].isnull() == False].reset_index()   
lenders_null = lenders.loc[lenders["country_code"].isnull() == True].reset_index()   

In [36]:
tot_notnull_users = len(lenders_notnull.index)
print(tot_notnull_users)

890539


Calcolo la distribuzione delle nazioni nel dataframe lenders_notnull

In [37]:
country_ripartition = lenders_notnull[["index", "country_code"]].groupby("country_code").count().reset_index().rename(columns = {"index":"n_users"})
country_ripartition["percentage"] = country_ripartition["n_users"]/tot_notnull_users*100

Ora riempio il dataframe lenders_null in modo che abbia la stessa distribuzione di lenders_notnull.
Per fare questo uso la funzione np.random.choice (inserendo come seed '1234')
In questo modo assegno la nazionalità in modo randomico e non sistematico (x. es. partendo dall'alto), ottenendo la distribuzione finale che voglio ottenere
ma per fare questo devo normalizzare le percentuali dividendole per la loro somma, altrimenti otterei l'errore (probabilities do not sum to 1

In [38]:
country_ripartition["percentage"] /= country_ripartition["percentage"].sum()

In [39]:
np.random.seed(1234)
lenders_null["country_code"] = np.random.choice(country_ripartition["country_code"], size=len(lenders_null.index), p = country_ripartition["percentage"])

In [40]:
lenders = pd.concat([lenders_notnull, lenders_null]).drop(columns="index")

Ora che il dataframe è sistemato con la distribuzione giusta, proseguo con la richiesta dell'esercizio.

Integro i dataframe "lenders", il quale ha i nomi dei lenders e la loro nazionalità, e il dataframe "lenders_overall_lent" che indica per ogni lender l'ammontare "lent".

A quel punto raggruppo per nazionalità e effettuo una somma, ottenendo l'ammontare "lent" per ogni nazione.

Inserisco queste info nel dataframe "tot_lent_country".


In [41]:
tot_lent_country = pd.merge(lenders_overall_lent, lenders[["permanent_name", "country_code"]], left_on="lenders", right_on="permanent_name")
tot_lent_country = tot_lent_country.drop(columns="permanent_name")
tot_lent_country = tot_lent_country.groupby("country_code")["amount_per_person"].sum()
tot_lent_country = tot_lent_country.to_frame().rename(columns={"amount_per_person" : "lent_amount"}).reset_index()

Nel dataframe "tot_lent_country" ho indicato il country_code, mentre nel dataframe "tot_borr_country" ho il nome della nazione, informazioni che mi servono entrambe. Eseguo quindi un merge per averle entrambe a portata di mano.

In [42]:
country_stats = pd.read_csv(country_stats_path)
loans_country = pd.merge(borr_country, country_stats[["country_name", "country_code"]], on="country_name")

Unisco ora le informazioni riguardo ai money_lent e ai money_borrowed in modo da poter eseguire la differenza tra questi due valori.

In [43]:
country_lent_borr = pd.merge(tot_lent_country, loans_country, on="country_code")
country_lent_borr = country_lent_borr[["country_name", "country_code", "lent_amount", "borrowed_amount"]]
country_lent_borr["difference"] = (country_lent_borr["lent_amount"] - country_lent_borr["borrowed_amount"])

In [44]:
#Decommentare questa riga per vedere i risultati
#country_lent_borr.head()

**10) Which country has the highest ratio between the difference computed at the previous point and the population?**

Per risolvere questo punto ho già tutte le informazioni che mi servono a portata di mano, le integro facendo un merge e poi svolgo una semplice query. Visto che ottengo differenze negative, sto attento a prendere il valore assoluto dell'indice costruito.

In [45]:
country_lent_borr  = pd.merge(country_lent_borr, country_stats[["country_code", "population"]], on="country_code")
country_lent_borr["ratio"] = abs(country_lent_borr["difference"]/country_lent_borr["population"])
max = country_lent_borr["ratio"].idxmax()
country_max = country_lent_borr.loc[max]

In [46]:
#Decommentare questa riga per vedere i risultati
country_max

country_name           Armenia
country_code                AM
lent_amount            1433.87
borrowed_amount    2.05793e+07
difference        -2.05778e+07
population             2930450
ratio                  7.02208
Name: 2, dtype: object

**11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?**

Cerco per prima cosa quali sono i paesi sopra la soglia della povertà. Avendo solo pochi valori nulli, per non incasinare troppo il calcolo decido di levare tutti i valori nulli. Calcolo infine il ratio sopra la soglia di povertà e lo inserisco nel dataframe. Attraverso il metodo idxmax trovo il massimo.

In [47]:
country_stats_not_null = country_stats[country_stats["population_below_poverty_line"].isnull() == False]
#country_stats_not_null = country_stats["population_below_poverty_line"].dropna(axis = 0, how = any)

In [48]:
pd.options.mode.chained_assignment = None 
country_stats_not_null["population_above_poverty_line"] = (country_stats_not_null["population"] -(country_stats_not_null["population"]*country_stats_not_null["population_below_poverty_line"]/100))

In [49]:
country_lent_borr  = pd.merge(country_lent_borr, country_stats_not_null[["country_code", "population_above_poverty_line"]], on="country_code")

In [50]:
country_lent_borr["ratio_above_poverty"] = country_lent_borr["difference"]/country_lent_borr["population_above_poverty_line"]

In [51]:
max = country_lent_borr["ratio_above_poverty"].idxmax()
country_lent_borr.loc[max]

country_name                     United States
country_code                                US
lent_amount                        4.05517e+07
borrowed_amount                    2.82160e+07
difference                         1.23357e+07
population                           324459463
ratio                                0.0380191
population_above_poverty_line      2.75466e+08
ratio_above_poverty                  0.0447811
Name: 64, dtype: object

**12)  For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.**

La prima cosa da fare per velocizzare i conti è suddividere il dataframe in due dataframe, il primo conterrà solo gli elementi per cui "disburse year" e "expiration year" si collocano nello stesso anno, il secondo in cui gli anni dei due attributi sono differenti. Questa suddivisione nasce dal fatto che il conto per gli elementi che si trovano nello stesso annoè notevolmente più facile.

Per prima cosa resetto gli indici del dataframe che ho già pronto e mi occupo di rimuovere il fuso orario e le ore poiché sono ininfluenti nel calcolo

In [52]:
loans_not_null = loans_not_null.reset_index()

loans_not_null["disburse_time"] = loans_not_null["disburse_time"].dt.tz_localize(None)
loans_not_null["planned_expiration_time"] = loans_not_null["planned_expiration_time"].dt.tz_localize(None)
loans_not_null["disburse_time"] = loans_not_null["disburse_time"].dt.normalize()
loans_not_null["planned_expiration_time"] = loans_not_null["planned_expiration_time"].dt.normalize()

Divido ora il dataframe nei due sottoinsiemi

In [53]:
loans_same_year= loans_not_null[loans_not_null["disburse_time"].dt.year == loans_not_null["planned_expiration_time"].dt.year][["loan_id" , "disburse_time", "planned_expiration_time","loan_amount"]]
loans_diff_year = loans_not_null[loans_not_null["disburse_time"].dt.year != loans_not_null["planned_expiration_time"].dt.year][["loan_id" ,"disburse_time", "planned_expiration_time","loan_amount"]]

Mi occupo prima del calcolo di quelli dello stesso anno

In [54]:
loans_same_year_tot = loans_same_year.groupby(loans_same_year["disburse_time"].dt.year)["loan_amount"].sum()
loans_same_year_tot = loans_same_year_tot.to_frame().reset_index()

In [55]:
#Decommentare la riga seguente per vedere i risultati
#loans_same_year_tot

Ora che quelli che hanno lo stesso anno sono stati sistemati mi occupo di fare i conti per il dataframe dei loans che hanno gli anni diversi. Ciò perché questi andranno poi sommati a quelli che hanno gli anni uguali.

In [57]:
#Ripeto le righe n=(anni di durata+1) volte, seguendo la logica di cui sopra

loans_diff_year = loans_diff_year.loc[loans_diff_year.index.repeat(loans_diff_year.planned_expiration_time.dt.year - loans_diff_year.disburse_time.dt.year + 1)]
#Creo la colonna start_time

loans_diff_year["start_time"] = loans_diff_year.drop_duplicates()["disburse_time"].dt.year.apply(lambda x: datetime.datetime(x, 1, 1)) 
#Vado a modificarla incrementando di un anno ogni volta che la riga si ripete, per ottenere il dataframe che mi serve 

y = loans_diff_year["disburse_time"].dt.year
loans_diff_year["start_time"] = pd.to_datetime(loans_diff_year.groupby(loans_diff_year["loan_id"]).cumcount() + y, format='%Y')
#Creo end_time e la popolo similmente a start_time

loans_diff_year["end_time"] = pd.to_datetime(loans_diff_year.groupby(loans_diff_year["loan_id"]).cumcount() + y, format='%Y')
loans_diff_year["end_time"] = loans_diff_year["start_time"].dt.year.apply(lambda x: datetime.datetime(x, 12, 31)) 

In [58]:
#df.loc[condizioni, "days_to_end"] = assegno valore

In [59]:
#Prima resetto l'indice

loans_diff_year = loans_diff_year.reset_index()
#Implemento la logica che ho descritto poco sopra

loans_diff_year.loc[(loans_diff_year["planned_expiration_time"].dt.year > loans_diff_year["end_time"].dt.year) & (loans_diff_year["disburse_time"].dt.year == loans_diff_year["start_time"].dt.year), "days_to_end"] = (loans_diff_year["end_time"] - loans_diff_year["disburse_time"]).dt.days +1                       

#Implemento la logica che ho descritto poco sopra

loans_diff_year.loc[(loans_diff_year["planned_expiration_time"].dt.year > loans_diff_year["end_time"].dt.year) & (loans_diff_year["disburse_time"].dt.year != loans_diff_year["start_time"].dt.year), "days_to_end"] = (loans_diff_year["end_time"] - loans_diff_year["start_time"]).dt.days +1                                           

loans_diff_year.loc[(loans_diff_year["planned_expiration_time"].dt.year == loans_diff_year["end_time"].dt.year), "days_to_end"] = (loans_diff_year["planned_expiration_time"] - loans_diff_year["start_time"]).dt.days +1    

In [60]:
loans_diff_year["tot_days"] = (loans_diff_year["planned_expiration_time"] - loans_diff_year["disburse_time"]).dt.days +1  
loans_diff_year["loan_amount_per_year"] = (loans_diff_year["loan_amount"]* loans_diff_year["days_to_end"])/loans_diff_year["tot_days"]
loans_diff_year_tot = loans_diff_year.groupby(loans_diff_year["start_time"].dt.year)["loan_amount_per_year"].sum().to_frame().reset_index()

In [62]:
#Decommentare la riga seguente per vedere i risultati
#loans_diff_year_tot

Ora che ho i due dataframe pronti faccio un merge in modo da avere le colonne affiancate e le sommo, dei valori nulli che avevo prima devo fare in modo di sostituire i NaN con 0 in modo che la somma sia un numero e non un NaN.

In [64]:
tot = pd.merge(loans_diff_year_tot, loans_same_year_tot, left_on ="start_time", right_on="disburse_time", how="outer")
#tot

In [65]:
tot[["loan_amount_per_year" , "loan_amount"]] = tot[["loan_amount_per_year" , "loan_amount"]].fillna(0)
tot["start_time"] = tot["start_time"].fillna(tot["disburse_time"])
tot["disburse_time"] = tot["disburse_time"].fillna(tot["start_time"])


In [66]:
tot["total_per_year"] = tot["loan_amount_per_year"] + tot["loan_amount"]
tot = tot[["start_time", "total_per_year"]]

In [68]:
#Decommentare questa riga per vedere i risultati.
#tot